In [1]:
!pip install dgl
from os import listdir
import dgl
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 3.0MB 3.5MB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
epochs = int(input("Enter number of epochs: "))
learning_rate = float(input("Enter learning rate: "))

Enter number of epochs: 250
Enter learning rate: 0.01


In [4]:
filename = "/content/drive/My Drive/bigblue1.nets"

edge_file = "/content/add_edges_2.py"
edge_file_handle = open(edge_file, "w+")

node_count = 0

with open(filename,encoding="utf8") as f:
    netSection = False
    edges = 0
    lineCount = 0
    src=[]
    dest=[]
    for line in f:
        lineCount += 1
        #Newline
        if len(line) < 2:
            continue
            
        # Comments
        if line[0] == '#':
            continue
        #print(line)
        splitLine = line.split()
        #print("Words = ", len(splitLine))
        if(splitLine[0] == 'NetDegree'):
            
            if edges > 0: #Previous NetDegree not complete!
                print("Missed an edge at line: ", lineCount-1)
                
            edges = int(splitLine[2])
            netSection = True
            src=[]
            dest=[]
            #G.add_nodes(num)
            
        elif netSection == True:
          node_name = splitLine[0][1:]
          node_val  = int(node_name)
          if node_count < node_val+1:
              node_count = node_val+1
          
          if(splitLine[1] == 'O'):
              src.append(node_name)
              edges -= 1
          elif(splitLine[1] == 'I'):
              dest.append(node_name)
              edges -= 1
      
        if netSection == True:
            if edges == 0:
                if (len(src)<1):
                    print("No source edge, skipping")
                    continue
                if (len(dest)<1):
                    print("No dest edge, skipping")
                    continue
                    
                netSection = False
                for des_idx in range(len(dest)):
                    cmd = "G.add_edges("+src[0]+","+dest[des_idx]+")"+"\n"
                    edge_file_handle.write(cmd)
                    
        #if lineCount > 100:
        #    break


edge_file_handle.close()


No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No dest edge, skipping
Missed an edge at line:  90029
No dest edge, skipping
Missed an edge at line:  90050
No dest edge, skipping
Missed an edge at line:  90066
No dest edge, skipping
Missed an edge at line:  90089
No dest edge, skipping
Missed an edge at line:  90111
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No dest edge, skipping
Missed an edge at line:  90181
No dest edge, skipping
Missed an edge at line:  90197
No dest edge, skipping
Missed an edge at line:  90214
No dest edge, skipping
Missed an edge at line:  90225
No dest edge, skipping
Missed an edge at line:  90240
No dest edge, skipping
Missed an edge at line:  90251
No des

In [32]:
G = dgl.DGLGraph()
G.add_nodes(node_count)
exec(open(edge_file).read())
print("Nodes = ", G.number_of_nodes())
print("Edges = ", G.number_of_edges())

Nodes =  278164
Edges =  856251


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
embed = nn.Embedding(G.number_of_nodes(),6)

In [0]:
G.ndata['feat'] = embed.weight

In [0]:
from dgl.nn.pytorch import GraphConv

In [0]:
class GCN(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(GCN,self).__init__()
        self.conv1 = GraphConv(input_size,hidden_size)
        self.conv2 = GraphConv(hidden_size,num_classes)
    
    def forward(self,G,inputs):
        h = self.conv1(G,inputs)
        h = torch.relu(h) 
        h = self.conv2(G,h)
        return h

In [0]:
net = GCN(6,5,20)

In [0]:
inputs = embed.weight
a = np.linspace(0,G.number_of_nodes()-1,20,dtype=int)
labelled_nodes = torch.tensor(a)
labels = torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

# Training Model

In [40]:
import itertools
import time

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=learning_rate)
all_logits = []

start_time = time.time()

for epoch in range(epochs):
    logits = net(G, inputs)
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labelled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch {} | Loss: {}'.format(epoch + 1, loss.item()))

end_time = time.time()
print("Training Time: {}".format(end_time - start_time))

Epoch 1 | Loss: 3.0132768154144287
Epoch 2 | Loss: 2.9956512451171875
Epoch 3 | Loss: 2.978658676147461
Epoch 4 | Loss: 2.962378740310669
Epoch 5 | Loss: 2.9462552070617676
Epoch 6 | Loss: 2.9300923347473145
Epoch 7 | Loss: 2.914113759994507
Epoch 8 | Loss: 2.897691249847412
Epoch 9 | Loss: 2.8804798126220703
Epoch 10 | Loss: 2.862473964691162
Epoch 11 | Loss: 2.8436503410339355
Epoch 12 | Loss: 2.8240396976470947
Epoch 13 | Loss: 2.803389549255371
Epoch 14 | Loss: 2.7817158699035645
Epoch 15 | Loss: 2.7592132091522217
Epoch 16 | Loss: 2.736016273498535
Epoch 17 | Loss: 2.7120614051818848
Epoch 18 | Loss: 2.6873779296875
Epoch 19 | Loss: 2.662203311920166
Epoch 20 | Loss: 2.6368215084075928
Epoch 21 | Loss: 2.6113762855529785
Epoch 22 | Loss: 2.585618019104004
Epoch 23 | Loss: 2.560107707977295
Epoch 24 | Loss: 2.534532070159912
Epoch 25 | Loss: 2.5089070796966553
Epoch 26 | Loss: 2.4830381870269775
Epoch 27 | Loss: 2.456730604171753
Epoch 28 | Loss: 2.4299097061157227
Epoch 29 | Loss:

In [0]:
pos = {}
cluster_nodes = {} # Every value in key->value pair includes the cluster centre node as well.
for v in range(G.number_of_nodes()):
  pos[v] = all_logits[99][v].numpy()
  cls = pos[v].argmax()
  if cluster_nodes.get(cls) :
    cluster_nodes[cls].append(v)
  else:
    cluster_nodes[cls] = [] 
    cluster_nodes[cls].append(v)

In [42]:
print(cluster_nodes)

{0: [0, 24, 125, 128, 155, 216, 238, 248, 275, 282, 345, 372, 379, 385, 414, 420, 423, 448, 449, 466, 473, 485, 514, 520, 531, 544, 616, 617, 626, 627, 645, 646, 674, 675, 678, 679, 680, 720, 743, 779, 793, 798, 800, 845, 858, 907, 910, 911, 953, 977, 980, 1093, 1113, 1131, 1144, 1193, 1229, 1230, 1260, 1273, 1300, 1309, 1345, 1376, 1385, 1398, 1406, 1421, 1422, 1467, 1468, 1473, 1474, 1475, 1476, 1484, 1486, 1522, 1523, 1524, 1525, 1533, 1546, 1547, 1564, 1565, 1592, 1593, 1609, 1610, 1646, 1647, 1650, 1651, 1660, 1661, 1688, 1689, 1792, 1819, 1829, 1935, 1945, 1948, 1966, 1979, 1986, 1987, 2188, 2201, 2224, 2226, 2234, 2236, 2248, 2261, 2274, 2282, 2287, 2301, 2346, 2377, 2399, 2410, 2411, 2429, 2430, 2439, 2440, 2444, 2445, 2448, 2449, 2450, 2459, 2460, 2462, 2463, 2489, 2498, 2517, 2669, 2676, 2677, 2689, 2707, 2727, 2785, 2793, 2803, 2804, 2812, 2813, 2817, 2827, 2831, 2938, 2998, 3001, 3007, 3016, 3032, 3056, 3062, 3071, 3111, 3112, 3137, 3141, 3198, 3200, 3235, 3236, 3253, 3264,

# Cluster Quality Metric #2 : Number of Nodes per Cluster Distribution

In [0]:
def number_of_nodes_per_cluster():
  number_dict = {} # Dictionary mapping the node label to the number of nodes in a particular cluster
  for key,value in cluster_nodes.items():
    number_dict[key] = len(value)

  number_dict = sorted(number_dict.items()) # Sorting dictionary by keys in increasing order

  for ix in number_dict:
    print("Number of Nodes in Cluster {} = {}\n".format(ix[0],ix[1]))

  return number_dict

In [44]:
number_dict = number_of_nodes_per_cluster()

Number of Nodes in Cluster 0 = 7622

Number of Nodes in Cluster 1 = 10390

Number of Nodes in Cluster 2 = 29633

Number of Nodes in Cluster 3 = 4866

Number of Nodes in Cluster 4 = 1515

Number of Nodes in Cluster 5 = 4562

Number of Nodes in Cluster 6 = 28479

Number of Nodes in Cluster 7 = 35517

Number of Nodes in Cluster 8 = 5583

Number of Nodes in Cluster 9 = 583

Number of Nodes in Cluster 10 = 8941

Number of Nodes in Cluster 11 = 6651

Number of Nodes in Cluster 12 = 10138

Number of Nodes in Cluster 13 = 1018

Number of Nodes in Cluster 14 = 13915

Number of Nodes in Cluster 15 = 350

Number of Nodes in Cluster 16 = 51084

Number of Nodes in Cluster 17 = 30952

Number of Nodes in Cluster 18 = 80

Number of Nodes in Cluster 19 = 26285



# HashMap (Dictionary) for easy visualization of networkx graph
- Dictionary to map node numbers in a particular cluster to a standard count procedure of 0 to (n-1) 
- n = number of nodes in a particular cluster
- Cluster label is taken as input from user

In [0]:
num = int(input("Enter cluster number: "))

Enter cluster number: 10


In [0]:
standard_nodes_dict = {}

count = 0

for node in cluster_nodes[num]:
  standard_nodes_dict[node] = count
  count += 1

In [0]:
print(standard_nodes_dict.items())

dict_items([(44, 0), (45, 1), (48, 2), (49, 3), (88, 4), (89, 5), (90, 6), (137, 7), (140, 8), (192, 9), (193, 10), (194, 11), (200, 12), (201, 13), (202, 14), (232, 15), (233, 16), (234, 17), (256, 18), (257, 19), (258, 20), (263, 21), (280, 22), (281, 23), (282, 24), (284, 25), (285, 26), (286, 27), (296, 28), (297, 29), (298, 30), (303, 31), (345, 32), (346, 33), (347, 34), (357, 35), (358, 36), (359, 37), (387, 38), (388, 39), (389, 40), (396, 41), (397, 42), (398, 43), (443, 44), (459, 45), (462, 46), (503, 47), (504, 48), (505, 49), (539, 50), (540, 51), (541, 52), (557, 53), (558, 54), (559, 55), (563, 56), (564, 57), (565, 58), (623, 59), (629, 60), (638, 61), (682, 62), (683, 63), (704, 64), (713, 65), (725, 66), (739, 67), (802, 68), (805, 69), (806, 70), (849, 71), (931, 72), (933, 73), (1060, 74), (1061, 75), (1068, 76), (1069, 77), (1113, 78), (1132, 79), (1135, 80), (1152, 81), (1156, 82), (1236, 83), (1242, 84), (1295, 85), (1296, 86), (1301, 87), (1302, 88), (1321, 89),

# Clustering Visualisation: Cluster Quality Metric # 3
- Visualisation for any generic cluster label taken as input from user

In [0]:
# dict_keys = list(standard_nodes_dict.keys())

# input_edges = G.in_edges(dict_keys, form='uv')

# ax = input_edges[0].numpy()

# ax = [i for i in ax if i in standard_nodes_dict.keys()]

# print(ax)

In [0]:
src = []
dst = []

In [0]:
def build_graph():

  dict_keys = list(standard_nodes_dict.keys())

  input_edges = G.in_edges(dict_keys, form='uv')

  src = input_edges[0].numpy()
  dst = input_edges[1].numpy()

  m = src.shape[0]
  src1 = []
  dst1 = []

  for ix in range(m):
    if src[ix] not in standard_nodes_dict.keys():
      continue
    else:
      src1.append(src[ix])
      dst1.append(dst[ix])

  src1 = np.array(src1)
  dst1 = np.array(dst1)

  j = 0
  for num in src1:
    num = standard_nodes_dict.get(num)
    src1[j] = num
    j += 1

  i = 0
  for num1 in dst1:
      num1 = standard_nodes_dict[num1]
      dst1[i] = num1
      i += 1

  u = np.concatenate([src1,dst1])
  v = np.concatenate([dst1,src1])

  return dgl.DGLGraph((u,v))

In [0]:
G1 = build_graph()

In [0]:
print("Nodes = {}".format(G1.number_of_nodes()))
print("Edges = {}".format(G1.number_of_edges()))

Nodes = 14718
Edges = 14738


In [0]:
nx_G1 = G1.to_networkx().to_undirected()

In [0]:
# nx.write_edgelist(nx_G1,"/content/cluster14.csv")

In [0]:
# pos1 = nx.kamada_kawai_layout(nx_G1) # Session crashes on running this cell

In [0]:
# plt.style.use("seaborn")

In [0]:
# nx.draw(nx_G1,pos1,with_labels=True,node_color=[[0.7,0.7,0.7]])
# plt.show()

# Cluster Quality Metric # 1: Total number of edges between nodes which belong to different clusters divided by total edges represented as percentage (%)

In [45]:
m = 10 # Total number of clusters

edges_between_clusters = {}

for i in range(m):
  print("Calculating for cluster {}\n".format(i))
  nodes_i = cluster_nodes[i]

  for j in range(i+1,m):
    nodes_j = np.array(cluster_nodes[j])

    ij_input_edges = G.in_edges(nodes_i,form='uv')
    edges_src = np.array(ij_input_edges[0])
    
    count = 0

    for val1 in nodes_j:
      for val2 in edges_src:
        if val1 == val2:
          count += 1

    # for x in range(np.array(nodes_i).shape[0]):
    #   if edges_src[x] == nodes_j[x]:
    #     count += 1

    edges_between_clusters[(i,j)] = count

Calculating for cluster 0

Calculating for cluster 1

Calculating for cluster 2

Calculating for cluster 3

Calculating for cluster 4

Calculating for cluster 5

Calculating for cluster 6

Calculating for cluster 7

Calculating for cluster 8

Calculating for cluster 9



In [46]:
edges_between_clusters

{(0, 1): 277,
 (0, 2): 1970,
 (0, 3): 246,
 (0, 4): 117,
 (0, 5): 135,
 (0, 6): 1723,
 (0, 7): 1750,
 (0, 8): 539,
 (0, 9): 5,
 (1, 2): 4247,
 (1, 3): 667,
 (1, 4): 235,
 (1, 5): 563,
 (1, 6): 3902,
 (1, 7): 4320,
 (1, 8): 898,
 (1, 9): 61,
 (2, 3): 1425,
 (2, 4): 529,
 (2, 5): 1115,
 (2, 6): 9010,
 (2, 7): 9558,
 (2, 8): 2458,
 (2, 9): 135,
 (3, 4): 75,
 (3, 5): 191,
 (3, 6): 870,
 (3, 7): 1084,
 (3, 8): 225,
 (3, 9): 35,
 (4, 5): 49,
 (4, 6): 148,
 (4, 7): 258,
 (4, 8): 48,
 (4, 9): 9,
 (5, 6): 1131,
 (5, 7): 1581,
 (5, 8): 309,
 (5, 9): 29,
 (6, 7): 10187,
 (6, 8): 2688,
 (6, 9): 97,
 (7, 8): 3613,
 (7, 9): 152,
 (8, 9): 13}

In [0]:
percentage_dict = {}

for key,value in edges_between_clusters.items():
  value = (value/G.number_of_edges())*100
  value = round(value,4)
  percentage_dict[key] = value

In [48]:
percentage_dict

{(0, 1): 0.0324,
 (0, 2): 0.2301,
 (0, 3): 0.0287,
 (0, 4): 0.0137,
 (0, 5): 0.0158,
 (0, 6): 0.2012,
 (0, 7): 0.2044,
 (0, 8): 0.0629,
 (0, 9): 0.0006,
 (1, 2): 0.496,
 (1, 3): 0.0779,
 (1, 4): 0.0274,
 (1, 5): 0.0658,
 (1, 6): 0.4557,
 (1, 7): 0.5045,
 (1, 8): 0.1049,
 (1, 9): 0.0071,
 (2, 3): 0.1664,
 (2, 4): 0.0618,
 (2, 5): 0.1302,
 (2, 6): 1.0523,
 (2, 7): 1.1163,
 (2, 8): 0.2871,
 (2, 9): 0.0158,
 (3, 4): 0.0088,
 (3, 5): 0.0223,
 (3, 6): 0.1016,
 (3, 7): 0.1266,
 (3, 8): 0.0263,
 (3, 9): 0.0041,
 (4, 5): 0.0057,
 (4, 6): 0.0173,
 (4, 7): 0.0301,
 (4, 8): 0.0056,
 (4, 9): 0.0011,
 (5, 6): 0.1321,
 (5, 7): 0.1846,
 (5, 8): 0.0361,
 (5, 9): 0.0034,
 (6, 7): 1.1897,
 (6, 8): 0.3139,
 (6, 9): 0.0113,
 (7, 8): 0.422,
 (7, 9): 0.0178,
 (8, 9): 0.0015}